# Importación Librerías

In [2]:
import pandas as pd
from io import StringIO
import gdown
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ast
from collections import Counter
import numpy as np
import re
import time



# Metadata  - Google

## Importar los archivos desde la nube

tamaño:
carpeta: 2,76 Gb
dataframe 370 Mb
csv 2,3 Gb

In [16]:
## descargados directamente desde google drive

# IDs de los archivos de Google Drive
file_links = [
    'https://drive.google.com/file/d/1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS', 
    'https://drive.google.com/file/d/15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_', 
    'https://drive.google.com/file/d/1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz', 
    'https://drive.google.com/file/d/1Mj2oUZy5gGznhthcUGi8_sgKhBwypE74', 
    'https://drive.google.com/file/d/1IXok40Zp61CGwFDgyvLUwV02c4BWGrjj',
    'https://drive.google.com/file/d/1UmsN_ZOFQqVl7W9SbnxHkSQavo1_Iwqx', 
    'https://drive.google.com/file/d/1KfQBhJlnuziKjf-9haQGaiPtCPnUUDla', 
    'https://drive.google.com/file/d/1ebTUx2klGy7L9lGlZl3GCPXxSwSD55vX', 
    'https://drive.google.com/file/d/1td6twU60LAS-z5mB0MeSJEpGhH7jcGKm', 
    'https://drive.google.com/file/d/1NQgHgNm9PV8MSiOXNoQ2UkIF9e5AdLk7', 
    'https://drive.google.com/file/d/1GYwWfH7EvWMZn14vQRNr5CjEely4eWrB'
]

# Nombres de los archivos locales (presumiendo que siguen el patrón 1.json, 2.json, ..., 11.json)
file_names = [f'{i}.json' for i in range(1, len(file_links) + 1)]

# Inicializar una lista para almacenar los DataFrames
dataframes = []

# Descargar y leer cada archivo JSON
for file_link, file_name in zip(file_links, file_names):
    try:
        # Obtener el ID del archivo desde el enlace
        file_id = file_link.split('/d/')[1].split('/')[0]
        download_url = f'https://drive.google.com/uc?id={file_id}'
        
        # Descargar el archivo
        gdown.download(download_url, file_name, quiet=False)
        
        # Verificar si el archivo descargado es un JSON válido
        with open(file_name, 'r', encoding='utf-8') as file:
            first_char = file.read(1)
            if not first_char:
                raise ValueError(f"El archivo {file_name} está vacío.")
            file.seek(0)
        
        # Leer el archivo JSON en un DataFrame
        df = pd.read_json(file_name, lines=True)
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)
        
        # Eliminar el archivo descargado
        os.remove(file_name)
    
    except Exception as e:
        print(f"Error al procesar el archivo {file_name}: {e}")
        if os.path.exists(file_name):
            os.remove(file_name)

Downloading...
From (original): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS
From (redirected): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS&confirm=t&uuid=03f8792e-1011-4b2d-8196-607d15726cee
To: c:\Users\feder\Desktop\PF\1.json
100%|██████████| 256M/256M [00:06<00:00, 37.7MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_
From (redirected): https://drive.google.com/uc?id=15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_&confirm=t&uuid=ab4b6a9d-8aee-4153-9cc4-73dc4064ccae
To: c:\Users\feder\Desktop\PF\2.json
100%|██████████| 257M/257M [00:09<00:00, 27.3MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz
From (redirected): https://drive.google.com/uc?id=1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz&confirm=t&uuid=d854ef6a-e45c-47f1-b98a-9a99e26300d0
To: c:\Users\feder\Desktop\PF\3.json
100%|██████████| 259M/259M [00:07<00:00, 33.1MB/s] 
Downloading...
From (original): ht

In [17]:
del df

In [18]:
# Concatenar todos los DataFrames en uno solo
df_metadataGoog = pd.concat(dataframes, ignore_index=True)

In [21]:
del dataframes

## Filtrar por estado California

In [22]:
#extraer el estado de la dirección
df_metadataGoog['estado'] = df_metadataGoog['address'].str.extract(r', ([A-Z]{2}) \d{5}')
#filtro los datos de California, para liberar espacio
df_metadatosCA = df_metadataGoog[df_metadataGoog['estado'] == 'CA']
#elimino el dataframe que tiene los metadatos de todos los estados, para liberar memoria
del df_metadataGoog
#reseteo indice
df_metadatosCA.reset_index(inplace=True)
df_metadatosCA.drop('index', axis='columns', inplace=True)

## Armado de columnas

In [ ]:
#solo para usar cuando se quiere importar el archivo
#df_metadatosCA=pd.read_parquet('Archivos/metadatosCA.parquet', engine='pyarrow')

### Extraer Ciudad

In [9]:
# Función para extraer la ciudad
def extract_city(address):
    match = re.search(r',\s*([^,]+),\s*[A-Z]{2}\s*\d{5}', address)
    if match:
        return match.group(1).strip()
    return None

# Aplicar la función a la columna 'address' y crear la columna 'city'
df_metadatosCA['city'] = df_metadatosCA['address'].apply(extract_city)

### Hours

In [36]:
# Función para asegurarse de que el tiempo esté en el formato "HH:MMAM/PM"
def ensure_time_format(time_str):
    if '–' in time_str:
        parts = time_str.split('–')
        parts = [ensure_time_format(part) for part in parts]
        return '–'.join(parts)
    try:
        if ':' not in time_str:
            time_obj = pd.to_datetime(time_str, format='%I%p', errors='coerce')
        else:
            time_obj = pd.to_datetime(time_str, format='%I:%M%p', errors='coerce')
        if time_obj is not pd.NaT:
            return time_obj.strftime('%I:%M%p')
    except Exception as e:
        print(f"Error formatting time: {time_str}, error: {e}")
    return None

# Función para calcular las horas diurnas (8 AM - 10 PM)
def calculate_day_hours(hours_array):
    total_day_hours = 0
    if hours_array is not None and isinstance(hours_array, np.ndarray):
        for entry in hours_array:
            if isinstance(entry, np.ndarray) and len(entry) == 2:
                day, hours = entry
                if 'Closed' in hours:
                    continue
                if '–' in hours:
                    open_time, close_time = hours.split('–')
                    open_time = ensure_time_format(open_time)
                    close_time = ensure_time_format(close_time)
                    
                    if open_time and close_time:
                        open_hour = pd.to_datetime(open_time, format='%I:%M%p').hour
                        close_hour = pd.to_datetime(close_time, format='%I:%M%p').hour

                        if open_hour < 8:
                            open_hour = 8
                        if close_hour > 22:
                            close_hour = 22

                        if close_hour < open_hour:
                            close_hour += 24  # Para manejar el cambio de día

                        total_day_hours += max(0, close_hour - open_hour)
    return total_day_hours

# Función para calcular las horas nocturnas (10 PM - 8 AM)
def calculate_night_hours(hours_array):
    total_night_hours = 0
    if hours_array is not None and isinstance(hours_array, np.ndarray):
        for entry in hours_array:
            if isinstance(entry, np.ndarray) and len(entry) == 2:
                day, hours = entry
                if 'Closed' in hours:
                    continue
                if '–' in hours:
                    open_time, close_time = hours.split('–')
                    open_time = ensure_time_format(open_time)
                    close_time = ensure_time_format(close_time)

                    if open_time and close_time:
                        open_hour = pd.to_datetime(open_time, format='%I:%M%p').hour
                        close_hour = pd.to_datetime(close_time, format='%I:%M%p').hour

                        if close_hour < open_hour:
                            close_hour += 24  # Para manejar el cambio de día

                        night_hours = 0
                        if open_hour < 8:
                            night_hours += min(8, close_hour) - open_hour
                        if close_hour > 22:
                            night_hours += close_hour - 22

                        total_night_hours += max(0, night_hours)
    return total_night_hours

# Aplicar las funciones al DataFrame
df_metadatosCA['Hours_day'] = df_metadatosCA['hours'].apply(calculate_day_hours)
df_metadatosCA['Hours_night'] = df_metadatosCA['hours'].apply(calculate_night_hours)

### Categorias

In [40]:
# Expandir las listas en filas individuales
categorias_expandidas = df_metadatosCA['category'].explode()

# Contar las ocurrencias de cada categoría
conteo_categorias = categorias_expandidas.value_counts()

# Eliminar la serie que ya no se usa
del categorias_expandidas

In [58]:
conteo_categorias.to_csv('Archivos/categoriasCA.csv')

### Explotar columna MISC

In [51]:
# Función para extraer y expandir los diccionarios en nuevas columnas
def expand_misc_column(misc_dict):
    if pd.isna(misc_dict):
        return pd.Series()
    expanded = {}
    for key, value in misc_dict.items():
        if value is not None and isinstance(value, np.ndarray):
            expanded[key] = ', '.join(value)
        else:
            expanded[key] = value
    return pd.Series(expanded)

# Aplicar la función al DataFrame
expanded_df = df_metadatosCA['MISC'].apply(expand_misc_column)

# Unir el DataFrame original con el DataFrame expandido
df_metadatosCA = pd.concat([df_metadatosCA, expanded_df], axis=1)

#Eliminar la columna MISC que ya no se usa
df_metadatosCA.drop(columns='MISC', inplace=True)

del expanded_df

### Accesibility

In [77]:
df_metadatosCA['Accessibility']=='Wheelchair accessible entrance, Wheelchair accessible parking lot, Wheelchair accessible restroom, Wheelchair accessible seating'

0         False
1         False
2         False
3         False
4         False
          ...  
297288    False
297289    False
297290    False
297291    False
297292    False
Name: Accessibility, Length: 297293, dtype: bool

In [ ]:
df_metadatosCA['Activities'].value_counts()

In [56]:
df_metadatosCA['Accessibility'].value_counts()
df_metadatosCA['Activities'].value_counts()
df_metadatosCA['Amenities'].value_counts()
df_metadatosCA['Atmosphere'].value_counts()
df_metadatosCA['Crowd'].value_counts()
df_metadatosCA['Dining options'].value_counts()
df_metadatosCA['From the business'].value_counts()
df_metadatosCA['Getting here'].value_counts()
df_metadatosCA['Health & safety'].value_counts()
df_metadatosCA['Health and safety'].value_counts()
df_metadatosCA['Highlights'].value_counts()
df_metadatosCA['Offerings'].value_counts()
df_metadatosCA['Payments'].value_counts()
df_metadatosCA['Planning'].value_counts()
df_metadatosCA['Popular for'].value_counts()
df_metadatosCA['Recycling'].value_counts()
df_metadatosCA['Service options'].value_counts()

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'state',
       'relative_results', 'url', 'estado', 'city', 'Hours_day', 'Hours_night',
       'Accessibility', 'Activities', 'Amenities', 'Atmosphere', 'Crowd',
       'Dining options', 'From the business', 'Getting here',
       'Health & safety', 'Health and safety', 'Highlights', 'Offerings',
       'Payments', 'Planning', 'Popular for', 'Recycling', 'Service options'],
      dtype='object')

## Exportaciones

### Machine Learning

In [62]:
# Elegir ciudad
ciudadelegida='Los Angeles'

In [78]:
# Generar el dataframe a exportar
df_ML = df_metadatosCA.loc[df_metadatosCA['city'] == ciudadelegida, 
                           ['address', 'gmap_id', 'latitude', 'longitude',
                            'category', 'avg_rating', 'num_of_reviews', 'Hours_day', 'Hours_night']]

# Exportar el dataframe
df_ML.to_csv('Archivos/metadatos_ML.csv', index=False)

### EDA

In [28]:
#guardo el archivo parquet, para poder importarlo si es necesario
df_metadatosCA.to_parquet('Archivos/metadatosCA.parquet', engine='pyarrow')

# Reviews Estados - Google

## Importar datos (cambiar a la nube)

In [ ]:
# Leer el archivo y almacenar cada línea en una lista
lines = []

for i in range (18):
    file='reviews-estados/review-California/'+str(i+1)+'.json'
    with open(file, 'r') as file:
        for line in file:
            lines.append(line)

In [ ]:
# Unir todas las líneas en un solo string y leerlo como un DataFrame
all_lines = ''.join(lines)
df_reviews = pd.read_json(StringIO(all_lines), lines=True)

In [ ]:
del lines
del all_lines
del file
del line

Espacio que ocupan
Carpeta: 763 Mb (3,8 segundos en importar para una lista, 48,44 segundos para pasarlo a dataframe)
Dataframe: 164,8 Mb
CSV: 545 Mbs

# Business - YELP

## Importar los archivos desde la nube

In [10]:
# ID del archivo de Google Drive
file_id = '1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu'
# Construir la URL de descarga
download_url = f'https://drive.google.com/uc?id={file_id}'

# Descargar el archivo usando gdown
file_path = 'business.pkl'
gdown.download(download_url, file_path, quiet=False)

# Cargar el archivo pickle en un DataFrame de pandas
df_business = pd.read_pickle(file_path)

# Eliminar el archivo descargado
os.remove(file_path)

Downloading...
From (original): https://drive.google.com/uc?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu
From (redirected): https://drive.google.com/uc?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu&confirm=t&uuid=69cf883e-c12b-4736-b320-965493d099b8
To: c:\Users\feder\Desktop\PF\business.pkl
100%|██████████| 116M/116M [00:03<00:00, 36.9MB/s] 


## Transformación de los Datos 


In [5]:
# Elimina las columnas duplicadas
df = df_business.loc[:, ~df_business.columns.duplicated()]

#  Filtra negocios en un estado específico (por ejemplo, 'CA' para California)
business_california = df[df['state'] == 'CA']

# Elimina valores nulos en las columnas attributes y hours
business_california_sn= business_california.dropna(subset=['attributes', 'hours'])

# Modificación del tipo  de datos en las columnas 'latitude','longitud','stars','review_count','is-open'

business_california_sn['latitude'] = pd.to_numeric(business_california_sn['latitude'], errors='coerce')
business_california_sn['longitude'] = pd.to_numeric(business_california_sn['longitude'], errors='coerce')
business_california_sn['stars'] = pd.to_numeric(business_california_sn['stars'], errors='coerce')
business_california_sn['review_count'] = pd.to_numeric(business_california_sn['review_count'], errors='coerce')
business_california_sn['is_open'] = pd.to_numeric(business_california_sn['is_open'], errors='coerce')

In [14]:
#guardo el archivo parquet, para poder importarlo si es necesario
business_california_sn.to_parquet('Archivos/business_CA_Yelp', engine='pyarrow')

# Review - YELP


## Importación de los Datos 


para realiza la importacion de los datos de reviews se leyó el archivo por Chunks o bloques de 1000 lineas , luego se hizo un filtrado por el business_id , correspondiente 

In [ ]:
# Definir la ruta al archivo JSON
file_path = '/content/drive/My Drive/set_de_datos_Yelp/review (1).json'

# Definir el tamaño del chunk (número de líneas por chunk)
chunk_size = 1000  # Puedes ajustar este valor según sea necesario

# Lista para almacenar los DataFrames procesados
results = []

# Leer el archivo JSON en chunks
chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# Procesar cada chunk por separado
for i, chunk in enumerate(chunks):
    # Procesar el chunk como sea necesario
    print(f"Procesando chunk {i + 1}")

    # Ejemplo de procesamiento: mostrar las primeras filas del chunk
    print(chunk.head())

    # Guardar resultados intermedios en una lista
    results.append(chunk)

    # Liberar memoria si no necesitas almacenar los chunks completos
    del chunk

# Combinar todos los chunks procesados en un solo DataFrame final
final_df = pd.concat(results, ignore_index=True)



# Guardar el DataFrame final en un archivo CSV
intermediate_file_path = '/content/drive/My Drive/set_de_datos_Yelp/final_dataframe.csv'
final_df.to_csv(intermediate_file_path, index=False)

# Liberar memoria después de guardar
del final_df
del results
import gc
gc.collect()

# Ahora puedes cargar 'final_dataframe.csv' en cualquier momento para operar sobre él
loaded_df = pd.read_csv(intermediate_file_path)

# Ejemplo de operaciones sobre el DataFrame cargado
print(loaded_df.head())

In [ ]:
#lectura del archivo , en formato csv
intermediate_file_path = '/content/drive/My Drive/set_de_datos_Yelp/final_dataframe.csv'
loaded_df = pd.read_csv(intermediate_file_path)

# Ahora puedes cargar 'final_dataframe.csv' en cualquier momento para operar sobre él
ruta = '/content/drive/My Drive/set_de_datos_Yelp/business_california.xlsx'
business_df = pd.read_excel(ruta)

filtered_reviews = loaded_df[loaded_df['business_id'].isin(business_df['business_id'])]

### transformación

In [ ]:
# Calcular valores nulos y porcentajes
valores_nulos = filtered_reviews.isnull().sum()


# Contar las ocurrencias de cada fila duplicada
duplicate_counts = filtered_reviews[filtered_reviews.duplicated()].sum()



In [1]:


# Escribir el DataFrame como un archivo Parquet
df.to_parquet('Archivos/review_ca.parquet', index=False)


# USER - YELP

## Carga de datos

In [5]:

# lectura del archivo user,parquet
df = pd.read_parquet('D:/usuarios/CIN/MIS DOCUMENTOS/Proyecto_Final/set_de_datos_Yelp/user.parquet')
# lectura del archivo review_ca.parquet para filtrar solo los de CA y disminuir el tamaño del archivo
reviews_ca = pd.read_parquet('Archivos/review_ca.parquet')
#Filtrar las reseñas solo para los user_ID específicos del archivo csv
filtered_reviews_ca = df[df['user_id'].isin(reviews_ca['user_id'])]

## Transformación

In [9]:
# Calcular valores nulos y porcentajes
valores_nulos = filtered_reviews_ca.isnull().sum()
# Contar las ocurrencias de cada fila duplicada
duplicate_counts = filtered_reviews_ca[filtered_reviews_ca.duplicated()].sum()

In [10]:
# Escribir el DataFrame como un archivo Parquet
filtered_reviews_ca.to_parquet('Archivos/user_ca.parquet', index=False)